In [1]:
from yfpy.query import YahooFantasySportsQuery
from yfpy.models import YahooFantasyObject, Player, StatCategories
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import os
from nhlpy import NHLClient
from yfpy.utils import reformat_json_list, unpack_data, prettify_data, jsonify_data
import requests
import json
from zoneinfo import ZoneInfo
from collections import defaultdict
import unicodedata
from pathlib import Path
import xml.etree.ElementTree as ET
import math
from yahoo_oauth import OAuth2
from requests.adapters import HTTPAdapter, Retry

In [2]:
load_dotenv()
CLIENT_ID = os.getenv("YAHOO_CONSUMER_KEY")
CLIENT_SECRET = os.getenv("YAHOO_CONSUMER_SECRET")

oauth = OAuth2(
    CLIENT_ID,
    CLIENT_SECRET,
    browser_callback=True,
)

In [3]:
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=0.1,
    status_forcelist=[ 500, 502, 503, 504 ]
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("http://", adapter)
session.mount("https://", adapter)


In [4]:
num_requests = 0

In [5]:
def api_request(url):
    global num_requests
    num_requests += 1
    if not oauth.token_is_valid():
        oauth.refresh_access_token()

    headers = {
        "Authorization": f"Bearer {oauth.access_token}",
        "Content-Type": "application/json"
    }
    response = session.get(url, headers=headers)
    if response.status_code != 200:
        print(response)
        print(response.text)
        response.raise_for_status()
    xml = response.text
    data = xml_to_dict(response.text)["fantasy_content"]
    return data

In [6]:
nhl_client = NHLClient()

In [7]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427

In [8]:
def strip_namespace(tag):
    """Remove namespace from the tag name."""
    return tag.split("}")[-1] if "}" in tag else tag

def parse_element(element):
    parsed_data = {}
    children = list(element)

    # Check if all children have the same tag (implying a list)
    child_tags = [strip_namespace(child.tag) for child in children]
    is_list = len(set(child_tags)) == 1 and len(children) > 1

    # Convert child elements into dictionary keys
    if is_list:
        return [parse_element(child) for child in children]  # Return a list directly
    else:
        for child in children:
            tag = strip_namespace(child.tag)
            child_data = parse_element(child)

            # Handle multiple children with the same tag by storing them as lists
            if tag in parsed_data:
                if not isinstance(parsed_data[tag], list):
                    parsed_data[tag] = [parsed_data[tag]]
                parsed_data[tag].append(child_data)
            else:
                parsed_data[tag] = child_data

    # If the element has text content, add it
    text = element.text.strip() if element.text else ""
    if text and not parsed_data:
        return text  # Return text if no nested structure

    return parsed_data or text  # Return text if no nested structure

def xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    return {strip_namespace(root.tag): parse_element(root)}

In [9]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [10]:
def get_league():
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key};out=standings"
    resp_data = api_request(url)["league"]
    return resp_data

In [11]:
league = get_league()
teams = league["standings"]["teams"]
teams

[{'team_key': '427.l.97108.t.2',
  'team_id': '2',
  'name': 'Eric’s a Bitch Team',
  'is_owned_by_current_login': '1',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/2',
  'team_logos': {'team_logo': {'size': 'large',
    'url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png'}},
  'waiver_priority': '4',
  'number_of_moves': '59',
  'number_of_trades': '0',
  'roster_adds': {'coverage_type': 'week',
   'coverage_value': '26',
   'value': '0'},
  'clinched_playoffs': '1',
  'league_scoring_type': 'headpoint',
  'draft_position': '4',
  'has_draft_grade': '0',
  'managers': {'manager': {'manager_id': '2',
    'nickname': 'Ken',
    'guid': 'TPMH3J2R4LWWFJV4GPPLVJPZYI',
    'is_current_login': '1',
    'email': 'dragonkenkj@gmail.com',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg',
    'felo_score': '656',
    'felo_tier': 'silver'}},
  'team_stats': {'coverage_type': 'season',
   'season': '2023',
   'stats': [{'stat_id': '1

In [12]:
def get_team_name(team_key):
    team_name = next(team["name"] for team in teams if team["team_key"] == team_key)
    return team_name

In [13]:
def get_all_team_rosters_by_date(date):
    """
    Parameters:
    date (str): Date to get roster for in YYYY-MM-DD format

    Returns:
    List of teams with the roster details for that date
    (does not include stats, need to make a separate reequest to get stats for all these players)
    Trieed everything, can't get stats if you're getting data for multiple teams
    f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players/stats;type=date;date={date}" doesn't work
    """
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players" # TODO: Would it be better to store team_keys once and make request to teams endpoint instad of league?
    resp_data = api_request(url)["league"]["teams"]
    return resp_data

In [14]:
def get_stats_for_players_by_date(player_keys, date):
    """
    Parameters:
    player_keys (str[]): All player keys
    date (str): Date to get roster for in YYYY-MM-DD format
    """
    league_key = f"{game_id}.l.{league_id}"
    data = []
    for i in range(math.ceil(len(player_keys)/25)):
        player_keys_str = ",".join(player_keys[i*25:min((i+1)*25, len(player_keys))])
        url = f"{BASE_URL}/league/{league_key}/players;player_keys={player_keys_str}/stats;type=date;date={date};start={i*25}" # Need to access league endpoint to get player_points attribute, otherwise calculate TODO: Is it faster to calculate it ourselves or request more information from the API?
        resp_data = api_request(url)
        data += resp_data["league"]["players"]
    return data

In [15]:
def flatten_list_of_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

In [16]:
"""
Things to track
- Points missed out on
- Number of times roster wasn't optimized
"""
def get_bench_oppurtunity_cost(bench_players, team_roster, players_points):
    def get_replacement_players(bench_player):
        replacement_positions = bench_player["display_position"].split(",")
        replacement_players = []
        for position in replacement_positions:
            players_matching_position = [player for player in team_roster if player["selected_position"]["position"] == position and player["player_key"] not in selected_replacements]
            new_positions = flatten_list_of_lists([player["display_position"].split(",") for player in players_matching_position])
            new_positions_unique = [position for position in new_positions if position not in replacement_positions]
            replacement_positions += new_positions_unique
            replacement_players += players_matching_position
        return replacement_players

    selected_replacements = []
    points_missed_out = 0
    num_wrong_bench = 0
    for bench_player in sorted(bench_players, key=lambda player: players_points[player["player_key"]], reverse=True):
        replacement_players = get_replacement_players(bench_player)
        if len(replacement_players) == 0:
            num_wrong_bench += 1
            points_missed_out += players_points[bench_player["player_key"]] - players_points.get(worst_replacement_player["player_key"], 0)
            selected_replacements.append(worst_replacement_player["player_key"])
        else:
            worst_replacement_player = sorted(replacement_players, key=lambda player: players_points.get(player["player_key"], 0))[0]
            if players_points.get(worst_replacement_player["player_key"], 0) < players_points[bench_player["player_key"]]:
                num_wrong_bench += 1
                points_missed_out += players_points[bench_player["player_key"]] - players_points.get(worst_replacement_player["player_key"], 0)
                selected_replacements.append(worst_replacement_player["player_key"])
    num_benched = len(bench_players)
    return points_missed_out, num_wrong_bench, num_benched

In [17]:
"""
Selected Position Values: {'G', 'NA', 'IR+', 'D', 'BN', 'RW', 'LW', 'C'}
- Number of games played by team
- Points left on bench

Theodore's Poo Poo Hospital 1066
Eric’s a Bitch Team         1045
Cyrus's Cursed Team         1043
eric's Awe-Inspiring Team   1021
Tim's Terrific Team         1004
adam2                       950
Kevin's Incredible Team     907
Miami Steamrollers          775
"""
league = get_league()
league_start_date = datetime.strptime(league["start_date"], "%Y-%m-%d")
league_end_date = datetime.strptime(league["end_date"], "%Y-%m-%d")
games_by_team = defaultdict(int)
points_left_on_bench_by_team = defaultdict(list)
benches_by_team = defaultdict(lambda: {"points_missed_out": 0, "num_wrong_bench": 0, "num_benched": 0})
for i in range((league_end_date - league_start_date).days+1):
    print(i)
    current_date_str = (league_start_date + timedelta(days=i)).strftime("%Y-%m-%d")
    teams = get_all_team_rosters_by_date(current_date_str)
    player_keys = [player["player_key"] for team in teams for player in team["roster"]["players"]] # This needs to be done first so request can be batched
    players = get_stats_for_players_by_date(player_keys, current_date_str)
    players_points = {player["player_key"]: float(player["player_points"]["total"]) for player in players if player["player_stats"]["stats"][0]["value"] != "-"} # Filter out players that didn't play that day
    for team in teams:
        bench_players = []
        team_roster = team["roster"]["players"]
        for player in team_roster:
            if player["selected_position"]["position"] not in ["BN", "IR+", "NA"] and player["player_key"] in players_points:
                games_by_team[team["team_key"]] += 1
            elif player["selected_position"]["position"] in ["BN", "IR+", "NA"] and player["player_key"] in players_points:
                bench_players.append(player)
        points_missed_out, num_wrong_bench, num_benched = get_bench_oppurtunity_cost(bench_players, team_roster, players_points)
        benches_by_team[team["team_key"]]["points_missed_out"] += points_missed_out
        benches_by_team[team["team_key"]]["num_wrong_bench"] += num_wrong_bench
        benches_by_team[team["team_key"]]["num_benched"] += num_benched
games_by_team_sorted = sorted(games_by_team.items(), key=lambda item: item[1], reverse=True)
games_by_team_sorted = [[get_team_name(team_key), games] for team_key, games in games_by_team_sorted]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173


In [ ]:
# 1204 requests rate limit

In [22]:
num_requests

1204

In [18]:
for team_key, bench_stats in benches_by_team.items():
    print(f'{get_team_name(team_key)} {round(bench_stats["points_missed_out"]/bench_stats["num_benched"], 2)} {round(bench_stats["num_wrong_bench"]/bench_stats["num_benched"], 2)}')

adam2 3.31 0.77
Eric’s a Bitch Team 3.21 0.75
Theodore's Poo Poo Hospital 2.44 0.63
Miami Steamrollers 3.89 0.83
eric's Awe-Inspiring Team 3.0 0.69
Tim's Terrific Team 3.51 0.75
Cyrus's Cursed Team 3.15 0.64
Kevin's Incredible Team 3.76 0.85


In [ ]:
def get_league_game_weeks():
    game_weeks = query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/game/{game_id}/game_weeks", 
        ["game", "game_weeks"]
    )
    game_weeks_league = game_weeks
    game_weeks_league[league_start_week-1].start = league_start_date_str
    game_weeks_league[league_end_week-1].end = league_end_date_str
    return game_weeks_league

In [ ]:
league_game_weeks = get_league_game_weeks()

In [ ]:
def get_dates_by_week(week):
    start_date_str = league_game_weeks[week-1].start
    end_date_str = league_game_weeks[week-1].end
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    return [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

In [ ]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [ ]:
def get_top_n_players_by_position(n, position):
    if position == "F":
        position = "C,LW,RW"
    return [playere for i in range(int(n/25)+1) for player in query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;sort=PTS;sort_type=season;position={position};count={n};start={i*25}/stats",
        ["league", "players"]
    )]

In [ ]:
'''
Get's daily scores for each matchup of the week.
'''
def get_matchups_by_week(week):
    dates = get_dates_by_week(week)
    resp = query.get_response(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/scoreboard;week={week}/matchups/teams/stats_collection;types=date;date={','.join(dates)}", 
    )
    teams_dict = resp.json().get('fantasy_content', {}).get('league', [{}])[1].get('scoreboard', {}).get("0").get("matchups", {})
    teams_dict = unpack_data(teams_dict, YahooFantasyObject)
    teams_dict = [el["matchup"] for el in teams_dict] # Convert object to list
    return teams_dict

In [ ]:
def get_player_game_log_nhl(player_id):
    season = f"{league_info.season}{league_info.season+1}"
    return requests.get(f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2").json()["gameLog"] # 2 inndicates regular season games

In [ ]:
def get_game_info_nhl(game_id):
    return requests.get(f"https://api-web.nhle.com/v1/gamecenter/{game_id}/landing").json()


In [ ]:
def get_player_team_by_date(player_name: str, player_position: str, date: str):
    '''
    Parameters:
    player_name (str): Full name of player
    date (str): Date to search for in YYYY-MM-DD format

    Returns:
    str: 3 letter abbreviation for NHL team name
    '''
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in player_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    game_log_for_date = next([game_log for game_log in player_game_log if game_log["gameDate"] == date])
    print(game_log_for_date)
    # return team_name

In [ ]:
def get_player_team_on_date(player_game_log, date):
    game = next(iter([game for game in player_game_log if datetime.strptime(game["gameDate"], "%Y-%m-%d") <= datetime.strptime(date, "%Y-%m-%d")]), None)
    if game is None:
        team = player_game_log[-1]["teamAbbrev"]
    else:
        team = game["teamAbbrev"]
    return team

In [ ]:
game_logs_cache = {}

In [ ]:
def normalize_name(name):
    # Normalize the name to NFKD form and remove diacritics
    return ''.join(
        c for c in unicodedata.normalize('NFKD', name)
        if not unicodedata.combining(c)
    )

In [ ]:
def get_game_log_by_player(player_key, player_name, player_position):
    if player_key in game_logs_cache:
        return game_logs_cache[player_key]
    player_name_url = player_name.replace(" ", "%20").replace("-", "%20")
    players_resp = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players_resp if normalize_name(player["name"]) == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in player_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_id = player["playerId"]
    player_game_log = get_player_game_log_nhl(player_id)
    game_logs_cache[player_key] = player_game_log
    return player_game_log

In [ ]:
query.get_game_stat_categories_by_game_id(427)

In [ ]:
'''
NHL Team contributing most to each fantasay team
Doesn't work, player_points doesn't take into account whether player was benched that day:
https://fantasysports.yahooapis.com/fantasy/v2/team/427.l.97108.t.1/roster;date={date}/players/stats;type=date;date={date}
Miami Steamrollers          EDM 0.33
adam2                       TBL 0.26
eric's Awe-Inspiring Team   VAN 0.17
Theodore's Poo Poo Hospital FLA 0.16
Cyrus's Cursed Team         FLA 0.15
Kevin's Incredible Team     BOS 0.15
Tim's Terrific Team         VAN 0.13
Eric’s a Bitch Team         COL 0.12

Eric’s a Bitch Team         Nathan MacKinnon    0.12
adam2                       Nikita Kucherov     0.11
Miami Steamrollers          Evan Bouchard       0.11
Kevin's Incredible Team     David Pastrnak      0.11
Theodore's Poo Poo Hospital Connor McDavid      0.10
eric's Awe-Inspiring Team   J.T. Miller         0.10
Tim's Terrific Team         Auston Matthews     0.09
Cyrus's Cursed Team         Matthew Tkachuk     0.08

Tim's Terrific Team         1426
Cyrus's Cursed Team         1061
Theodore's Poo Poo Hospital 985
eric's Awe-Inspiring Team   827
Eric’s a Bitch Team         774
Miami Steamrollers          711
adam2                       599
Kevin's Incredible Team     433
'''
# TODO: Properly iterate through teams
HITS_STAT_ID = 31
hits_by_team = defaultdict(int)
for team in league_teams:
    team_points_by_nhl_team = defaultdict(float)
    team_points_by_player = defaultdict(float)
    for week in range(league_start_week, league_end_week + 1):
        week_end_date = get_dates_by_week(week)[-1]
        roster_players = query.query(f"https://fantasysports.yahooapis.com/fantasy/v2/team/{team.team_key}/roster;week={week}/players/stats;type=week;week={week}", ["team", "roster", "0", "players"])
        for player in roster_players:
            if not player.player_points.total:
                continue
            player_game_log = get_game_log_by_player(player.player_key, player.name.full, player.display_position)
            nhl_team = get_player_team_on_date(player_game_log, week_end_date)
            print(player)
            team_points_by_nhl_team[nhl_team] += player.player_points.total
            team_points_by_player[player.player_key] += player.player_points.total
            hits_by_team[team.team_key] += next(iter([stat.value for stat in player.player_stats.stats if stat.stat_id == HITS_STAT_ID]), 0)
    team_points_by_nhl_team = sorted(team_points_by_nhl_team.items(), key=lambda item: item[1], reverse=True)
    team_points_by_player = sorted(team_points_by_player.items(), key=lambda item: item[1], reverse=True)
    top_nhl_team = team_points_by_nhl_team[0][0]
    top_nhl_team_pct = round(team_points_by_nhl_team[0][1]/sum([row[1] for row in team_points_by_nhl_team]), 2)
    top_player = team_points_by_player[0][0]
    top_player_name = query.query(f"https://fantasysports.yahooapis.com/fantasy/v2/player/{top_player}", ["player"], Player).name.full
    top_player_pct = round(team_points_by_player[0][1]/sum([row[1] for row in team_points_by_player]), 2)
    print(team.name.decode('utf-8'), top_nhl_team, top_nhl_team_pct)
    print(team.name.decode('utf-8'), top_player_name, top_player_pct)
    print(team.name.decode('utf-8'), hits_by_team[team.team_key])

In [ ]:
# hits_by_team = sorted(hits_by_team.items(), key=lambda item: item[1], reverse=True)
test = [print(next(team.name.decode('utf-8') for team in league_teams if team.team_key == team_key), round(hits)) for (team_key, hits) in hits_by_team]

In [ ]:
'''
Most/least points after dropping a player on average
'''
transactions = query.get_league_transactions()
transactions_drop = [{"player": player, "timestamp": transaction.timestamp} for transaction in transactions for player in transaction.players if player.transaction_data.type == "drop"]
team_name = {}
points_by_team = defaultdict(list)
for transaction in transactions_drop:
    print(transaction)
    transaction_date = datetime.fromtimestamp(transaction["timestamp"], tz=ZoneInfo("America/New_York"))
    transaction_player = transaction["player"]
    team_key = transaction_player.transaction_data.source_team_key
    player_name = transaction_player.name.full
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in transaction_player.display_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    if len(player_game_log) == 0:
        print(f"{player_name} didn't play any games this season")
        continue
    # TODO: Consider case when someone is dropped due to injury
    # last_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() <= transaction_date.date()]
    # if len(last_game) == 0: # No games played before drop
    #     first_game = player_game_log[-1]
    #     print(f"{player_name} didn't play any games yet")
    #     continue
    # Next game could either be the same day as pickup or game after pickup
    next_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() >= transaction_date.date()]
    if len(next_game) == 0: # Didn't play the rest of the season
        print(f"{player_name} didn't play any more games")
        continue
    next_game_date = next_game[-1]["gameDate"]
    player_stats = query.get_player_stats_by_date(transaction_player.player_key, next_game_date)
    points_by_team[team_key].append(player_stats.player_points.total)
    team_name[team_key] = transaction_player.transaction_data.source_team_name
team_average = []
for team_key, team_points in points_by_team.items():
    team_average.append((team_name[team_key], round(sum(team_points)/len(team_points), 1)))
team_average = sorted(team_average, key=lambda x: x[1])
print(team_average)

In [ ]:
'''
Biggest comeback
TODO: Add the following line to bottom of Team class:
self.team_stats_collection: List[TeamPoints] = self._extracted_data.get("team_stats_collection", []) # ADDED
52.8s
'''
deficits = []
for week in range(league_start_week, league_end_week + 1):
    matchups = get_matchups_by_week(week)
    for matchup in matchups:
        # Comeback win can't happen without a winner
        if matchup.is_tied:
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_idx, team_l_idx = (0, 1) if winner_id == team_ids[0] else (1, 0)
        deficit = 0
        start_date = datetime.strptime(matchup.week_start, "%Y-%m-%d")
        end_date = datetime.strptime(matchup.week_end, "%Y-%m-%d")
        # Don't include the last day since the matchup is over
        for i in range((end_date - start_date).days):
            current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
            team_w_points = matchup.teams[team_w_idx].team_stats_collection[i]["team_points"].total
            team_l_points = matchup.teams[team_l_idx].team_stats_collection[i]["team_points"].total
            deficit = round(deficit + team_l_points - team_w_points, 1)
            if deficit > 0:
                deficits.append((deficit, matchup))
biggest_deficits = sorted(deficits, key=lambda x: x[0])
biggest_deficits = list({f"{matchup.week}.{matchup.winner_team_key}": (deficit, matchup) for (deficit, matchup) in biggest_deficits}.values())
biggest_deficits = sorted(biggest_deficits, reverse=True, key=lambda x: x[0])
print("Biggest comebacks")
for diff, matchup in biggest_deficits[:5]:
    print(f"{matchup.week} {matchup.teams[0].name} {matchup.teams[1].name} - {diff}")

In [ ]:
'''
Biggest draft busts/steals
'''
# Get draft results
draft_results = query.get_league_draft_results()
draft_players = get_players([draft_result.player_key for draft_result in draft_results])
draft_players
draft_players_by_pos = {"F": [], "D": [], "G": []}
positions_map = {"C": "F", "LW": "F", "RW": "F", "D": "D", "G": "G"}
for draft_player in draft_players:
    draft_players_by_pos[positions_map[draft_player.primary_position]].append(draft_player)
# Get the top players by position
top_players_by_pos = {
    "F": get_top_n_players_by_position(len(draft_players_by_pos["F"]), "F"), 
    "D": get_top_n_players_by_position(len(draft_players_by_pos["D"]), "D"), 
    "G": get_top_n_players_by_position(len(draft_players_by_pos["G"]), "G")
}
# Get differences between draft player and top player
diffs = []
for key in draft_players_by_pos:
    for draft_player, top_player in zip(draft_players_by_pos[key], top_players_by_pos[key]):
        diff = round(draft_player.player_points.total - top_player.player_points.total, 1)
        diffs.append((diff, draft_player))
biggest_diffs = sorted(diffs, reverse=True, key=lambda x: x[0])
smallest_diffs = sorted(diffs, key=lambda x: x[0])
print("Biggest draft steals")
for diff, player in biggest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")
print("Biggest draft busts")
for diff, player in smallest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")